In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
import json
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import RobertaModel, RobertaTokenizer


In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
# load data
import pandas as pd
# data = [{'name': 'vikash', 'age': 27}, {'name': 'Satyam', 'age': 14}]
# test_data = pd.DataFrame.from_dict(data, orient='columns')
test_data = pd.read_json('data/test.jsonl', lines=True)
dev_data = pd.read_json('data/test.jsonl', lines=True)
train_data = pd.read_json('data/test.jsonl', lines=True)
test_data.head(10)

,premise,choice1,choice2,question,label,idx
0,The man turned on the faucet.,The toilet filled with water.,Water flowed from the spout.,effect,1,0
1,The girl found a bug in her cereal.,She poured milk in the bowl.,She lost her appetite.,effect,1,1
2,The woman retired.,She received her pension.,She paid off her mortgage.,effect,0,2
3,I wanted to conserve energy.,I swept the floor in the unoccupied room.,I shut off the light in the unoccupied room.,effect,1,3
4,The hamburger meat browned.,The cook froze it.,The cook grilled it.,cause,1,4
5,I doubted the salesman's pitch.,I turned his offer down.,He persuaded me to buy the product.,effect,0,5
6,I decided to stay home for the night.,The forecast called for storms.,My friends urged me to go out.,cause,0,6
7,My eyes became red and puffy.,I was sobbing.,I was laughing.,cause,0,7
8,The flame on the candle went out.,I blew on the wick.,I put a match to the wick.,cause,0,8
9,The man drank heavily at the party.,He had a headache the next day.,He had a runny nose the next day.,effect,0,9


In [4]:
# test
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# test_sequence = "{" + "effect" + "}" + "I ran the ice cube under warm water."
test_sequence = "{"+ test_data.iloc[0]['question'] + "}" + test_data.iloc[0]['premise']
print("test_sequence is: ", test_sequence)
print(tokenizer(test_sequence))
print(tokenizer.tokenize(test_sequence))

test_sequence is:  {effect}The man turned on the faucet.
{'input_ids': [0, 45152, 26715, 24303, 133, 313, 1224, 15, 5, 856, 17527, 594, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['{', 'effect', '}', 'The', 'Ġman', 'Ġturned', 'Ġon', 'Ġthe', 'Ġf', 'auc', 'et', '.']


In [5]:
# test 2
test_sequence = "{"+ test_data.iloc[1]['question'] + "}" + test_data.iloc[1]['premise']
print("test_sequence is: ", test_sequence)
print(tokenizer(test_sequence))
print(tokenizer.tokenize(test_sequence))

test_sequence is:  {effect}The girl found a bug in her cereal.
{'input_ids': [0, 45152, 26715, 24303, 133, 1816, 303, 10, 13673, 11, 69, 25629, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['{', 'effect', '}', 'The', 'Ġgirl', 'Ġfound', 'Ġa', 'Ġbug', 'Ġin', 'Ġher', 'Ġcereal', '.']


In [6]:
print(test_data.shape[0])

100


In [38]:
from torch import nn
from transformers import RobertaConfig, RobertaModel

class OurRobertaCOPA(torch.nn.Module):
    def __init__(self):
        super(OurRobertaCOPA, self).__init__()
        # self.configuration = RobertaConfig()
        # self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        # self.l1 = RobertaModel(self.configuration)
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 5)
        # hidden_dim=32 for later trials.
        self.lstm = nn.LSTM(768, 32, 1, bias=False)
        self.output_layer = nn.Linear(768, 2)

    def forward(self, sequence_1, sequence_2):
        # Two input here
        token_1 = tokenizer(sequence_1)
        token_2 = tokenizer(sequence_2)
        output_1 = self.l1(input_ids=torch.tensor(token_1["input_ids"]).unsqueeze(0), attention_mask=torch.tensor(token_1["attention_mask"]).unsqueeze(0))
        output_2 = self.l1(input_ids=torch.tensor(token_2["input_ids"]).unsqueeze(0), attention_mask=torch.tensor(token_2["attention_mask"]).unsqueeze(0))
        # RobertaModel(RobertaConfig())

        # _, (hidden_rep_1, _) = self.lstm(output_1.unsqueeze(0))
        # _, (hidden_rep_2, _) = self.lstm(output_2.unsqueeze(0))
        # _, (hidden_rep_1, _) = self.lstm(output_1)
        # _, (hidden_rep_2, _) = self.lstm(output_2)

        hidden_rep_1 = torch.nn.ReLU()(self.pre_classifier(output_1[0]))
        hidden_rep_2 = torch.nn.ReLU()(self.pre_classifier(output_2[0]))
        
        hidden_rep = torch.cat((hidden_rep_1, hidden_rep_2), 1)
        output = self.output_layer(hidden_rep.squeeze(0))     
        return output

In [39]:
model = OurRobertaCOPA()
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OurRobertaCOPA(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,),

In [41]:
epochs = 4
ce = nn.CrossEntropyLoss()
softmax = nn.Softmax(dim=0)
optimizer = optim.SGD(model.parameters(), lr=0.1)

for j in range(epochs):
    print('### Epoch: ' + str(j+1) + ' ###')
    av_loss = 0
    model.train()
    for i in range(0, train_data.shape[0]):

        x1 = "{"+ train_data.iloc[i]['question'] + "}" + train_data.iloc[i]['premise'] + train_data.iloc[i]['choice1']
        x2 = "{"+ train_data.iloc[i]['question'] + "}" + train_data.iloc[i]['premise'] + train_data.iloc[i]['choice2']
        y = torch.tensor(train_data.iloc[i]['label'])


        y_raw = model(x1, x2)
        y_hat = softmax(y_raw)

        loss = ce(y_hat, y)
        av_loss += loss

        loss.backward()

        optimizer.step()
    print(av_loss / train_data.shape[0])


### Epoch: 1 ###


ValueError: Expected input batch_size (41) to match target batch_size (0).